<a href="https://colab.research.google.com/github/LaurensSluyterman/PBPK-evaluation/blob/main/PBPK_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab contains all the necessary code in order to calcualte a confidence interval for a geometric mean ratio as is advocated in our paper  "*Towards more RobustEvaluation of the Predictive Performance of Physiologically Based Pharmacokinetic Models: using Confidence Intervals to support use of Model-informed Dosing in Clinical Care*", which is currently under review.

In [5]:
#@title The code that is used ot create the intervals (run this first).
import numpy as np
import scipy
from scipy import stats

def CI_I_LD(observations, predictions,
                                          alpha=0.1):
    """This function calculates a confidence interval using the individual
    predictions.

    The interval is calculated on logscale using two paired t-tests and then
    transformed back to the original scale. For more details on the reasoning
    behind the interval, please see our paper: ....

    Arguments:
      observations: An array containing the individual observations
      predictions: An array containing the individual predictions
      alpha: A float that determines the significance of the confidence interval.
        The significance is (1-alpha/2)*100%.

    Return:
      None
    """
    # Some preliminary checks
    if len(observations) != len(predictions):
      print('different number of predictions and observations')
      return
    if alpha > 1:
      print('alpha cannot be larger than 1. For a 90% confidence interval, use alpha=0.1')
    # Convert to logscale
    error_values = np.log(predictions) - np.log(observations)

    # Calcualte a CI on logscale for the mean of the error
    N = len(error_values)
    var = np.var(error_values)
    average = np.mean(error_values)
    t = scipy.stats.t(N-1).ppf(1-alpha/2)
    loglowerbound = average - t * np.sqrt(var / N)
    logupperbound = average + t * np.sqrt(var / N)

    # Convert the CI back to the original scale
    CI = [np.exp(loglowerbound), np.exp(logupperbound)]

    # Print the result
    print(f'GM-ratio: {np.exp(np.mean(error_values))}')
    print(f'{100*(1-alpha)}% confidence interval: {CI}')


def CI_G_LD(GM_observed, GM_predicted, GCV_observed, N, alpha=0.1):
    """This function calculates a confidence interval using the observed
       geometric mean, the predicted geometric mean, and the coefficient
       of variation.

    Arguments:
      GM_observed (float): The geometric mean of the observations
      GM_predicted (float): The geometric mean of the predictions
      GCV_observed (float): The geometric coefficient of variation. Note that values are
        often reported as percentiles in literature. If 70% is reported, use 0.7.
      N (int): The number of observations.
      alpha (float): Determines the significance of the confidence interval.
        The significance is (1-alpha/2)*100%.

    Return:
      None
    """
    average = np.log(GM_observed)
    var = np.log(GCV_observed**2 + 1)
    t = scipy.stats.t(N-1).ppf(1-alpha/2)
    loglowerbound = average + t * np.sqrt(var / N)
    logupperbound = average - t * np.sqrt(var / N)
    CI = [GM_predicted / np.exp(loglowerbound), GM_predicted / np.exp(logupperbound)]
    print(f'GM-ratio: {GM_predicted / GM_observed}')
    print(f'{100*(1-alpha)}% confidence interval: {CI}')

def CI_G_LD_small(GM_observed, GM_predicted, GCV_observed, GCV_predicted,
                  N_observed, N_predicted, alpha=0.1):
    """
    Calculate CI using the GLM approach and considering variance of predictions.

    This function calculates a CI of the difference in means on logscale and
    then transform this back to the original scale. A 2-sided unpaired
    t-test assuming unequal variances is used.

    Arguments:
        GM_observed: Observed geometric mean
        GM_predicted: Predicted geometric mean
        GCV_observed: Observed geometric coefficient of variation
        GCV_predicted: Predicted geometric coefficient of variation
        N_observed: Number of observed subjects
        N_predicted: Number of predicted subjects
        alpha: Confidence level. The defaul value of 0.1 corresponds to 90% CI.

    Returns:
        CI: The (1-alpha)*100% confidence interval.
    """
    if type(N_observed) != int:
      raise ValueError('N_observed must be an integer, do not use dots.')
    if type(N_predicted) != int:
      raise ValueError('N_observed must be an integer, do use dots.')
    average_observed = np.log(GM_observed)
    average_predicted = np.log(GM_predicted)
    var_observed = np.log(GCV_observed**2 + 1)
    var_predicted = np.log(GCV_predicted**2 + 1)
    var_total = var_observed / N_observed + var_predicted / N_predicted
    df = (var_observed / N_observed + var_predicted / N_predicted)**2 \
     / ((var_observed / N_observed)**2 / (N_observed - 1) + (var_predicted / N_predicted)**2 / (N_predicted - 1))
    t = scipy.stats.t(df).ppf(1-alpha/2)
    loglowerbound = average_predicted - average_observed - t * np.sqrt(var_total)
    logupperbound = average_predicted - average_observed + t * np.sqrt(var_total)
    CI = [np.exp(loglowerbound), np.exp(logupperbound)]
    print(f'GM-ratio: {GM_predicted / GM_observed}')
    print(f'{100*(1-alpha)}% confidence interval: {CI}')


The implementation of the code is give in the code block above. The above code block must be run first

## I-LD Approach: When individual level data is available

The cell below illustrates how a confidence interval can be created using individual observations and predicitons. The hypothetical values can be replaced by relevant values. By default, a 90% confidence interval is given. To get, for instance, a 95% confidence interval, change alpha to 0.05.


In [6]:
observations = [132,
                111,
                120,
                190,
                115,
                130,
                ]

predictions = [110,
               121,
               125,
               170,
               125,
               130,
               ]

try:
  CI_I_LD(observations, predictions, alpha=0.1)
except NameError:
  print('Run the first code block first to load the necessary functions.')


GM-ratio: 0.9862482242209722
90.0% confidence interval: [0.9080389449267313, 1.0711936588331088]


# G-LD appraoch large simulation

Typically, only the geometric mean and coefficient of variation are published and not the individual AUC values. In this case, the first approach is not possible and the second approach must be used.

In [9]:
GM_observed = 120
GM_predicted = 120
GCV_observed = 0.4 # N.B. 40% should be entered as 0.4!
Number_of_subjects = 20

try:
  CI_G_LD(GM_observed, GM_predicted, GCV_observed, N=Number_of_subjects, alpha=0.1)

except NameError:
  print('Run the first code block first to load the necessary functions.')

GM-ratio: 1.0
90.0% confidence interval: [0.8616065596990528, 1.1606225471975127]


# G-LD approach small simulation

If, due to computational constraints, it is infeasible to create a large simulation, the variance of the predictions can no longer be ignored. In this case, the following function should be used. Note that in this example, the interval is slightly wider than in the previous example since we only used 100 predictions. If we set N_predicted to 10.000, we a practically identical interval.

In [10]:
GM_observed = 120
GM_predicted = 120
GCV_observed = 0.4 # N.B. 40% should be entered as 0.4!
GCV_predicted = 0.8
N_observed = 20
N_predicted = 100


CI_G_LD_small(GM_observed=GM_observed, GM_predicted=GM_predicted, GCV_observed=GCV_observed,
              GCV_predicted=GCV_predicted, N_observed=N_observed, N_predicted=N_predicted, alpha=0.1)

GM-ratio: 1.0
90.0% confidence interval: [0.8298759846566229, 1.204999323379347]
